In [8]:
import PIL
from PIL import Image
import numpy as np
import os 
import random
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

from keras.wrappers.scikit_learn import KerasClassifier
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from keras import regularizers

from skopt import BayesSearchCV

In [9]:
dirs = ['/media/jan/Volume_big/datasets/Second_try/Grid_size_874_binary/3layer/no_porosity',
        '/media/jan/Volume_big/datasets/Second_try/Grid_size_874_binary/3layer/porosity']

length = 2000
image_paths = []
labels = []
data = []

In [10]:
height = 128
width = 128
depth = 3
inputShape = (height, width, depth)

In [11]:
for d in dirs: 
    cur_paths = []
    for path in os.listdir(d):
        full_path = os.path.join(d,path)
        cur_paths.append(full_path)
        
    random.seed(42)
    random.shuffle(cur_paths)
    cur_paths = cur_paths[:length]
    
    image_paths.append(cur_paths)        

image_paths = [val for sublist in image_paths for val in sublist]
random.seed(42)
random.shuffle(image_paths)

######
for path in image_paths:
    label = path.split(os.path.sep)[-2]
    labels.append(label)
    
    array = np.load(path)
    img = Image.fromarray(array)
    img = img.resize((height, width), PIL.Image.LANCZOS)
    array_resized_flattened = np.array(img)
    data.append(array_resized_flattened)
    
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
trainY = np.hstack((trainY, 1- trainY))

testY = lb.transform(testY)
testY = np.hstack((testY, 1- testY))

In [12]:
def create_model(lr = 0.001):
    
    model = Sequential()
    model.add(Conv2D(3,(6,6), padding="same", input_shape = inputShape))   #, kernel_regularizer=regularizers.l2(0.01)))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = -1))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(6, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=-1))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(32))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    
    model.add(Dense(2))
    model.add(Activation("sigmoid"))
    
    optimizer = Adam(lr = lr)
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model 

In [13]:
trainY.shape

(3000, 2)

In [14]:
trainY[1]

array([1, 0])

In [15]:
model = KerasClassifier(build_fn=create_model)

In [16]:
batch_size = [8, 16, 32]
epochs = [10,15,20]
learning_rate = [0.001, 0.01, 0.1]
param_grid = dict(batch_size=batch_size, epochs=epochs, lr = learning_rate)

In [17]:
param_grid

{'batch_size': [8, 16, 32], 'epochs': [10, 15, 20], 'lr': [0.001, 0.01, 0.1]}

In [25]:
class FixedBayesSearchCV(BayesSearchCV):
    """
    Dirty hack to avoid compatibility issues with sklearn 0.2 and skopt.
    Credit: https://www.kaggle.com/c/home-credit-default-risk/discussion/64004

    For context, on why the workaround see:
        - https://github.com/scikit-optimize/scikit-optimize/issues/718
        - https://github.com/scikit-optimize/scikit-optimize/issues/762
    """
    def __init__(self, estimator, search_spaces, optimizer_kwargs=None,
                n_iter=50, scoring=None, fit_params=None, n_jobs=1,
                n_points=1, iid=True, refit=True, cv=None, verbose=0,
                pre_dispatch='2*n_jobs', random_state=None,
                error_score='raise', return_train_score=False):
                """
                See: https://github.com/scikit-optimize/scikit-optimize/issues/762#issuecomment-493689266
                """

                # Bug fix: Added this line
                self.fit_params = fit_params

                self.search_spaces = search_spaces
                self.n_iter = n_iter
                self.n_points = n_points
                self.random_state = random_state
                self.optimizer_kwargs = optimizer_kwargs
                self._check_search_space(self.search_spaces)

                # Removed the passing of fit_params to the parent class.
                super(BayesSearchCV, self).__init__(
                        estimator=estimator, scoring=scoring, n_jobs=n_jobs, iid=iid,
                        refit=refit, cv=cv, verbose=verbose, pre_dispatch=pre_dispatch,
                        error_score=error_score, return_train_score=return_train_score)

    def _run_search(self, x):
        raise BaseException('Use newer skopt')

In [26]:
bayes = FixedBayesSearchCV(model, search_spaces=param_grid, n_iter = 2,random_state=1234)

In [27]:
bayes_result = bayes.fit(trainX, trainY)

Epoch 1/20
2400/2400 [==============================] - 3s 1ms/step - loss: 0.6682 - acc: 0.6500
Epoch 2/20
2400/2400 [==============================] - 2s 776us/step - loss: 0.5834 - acc: 0.7021
Epoch 3/20
2400/2400 [==============================] - 2s 784us/step - loss: 0.5702 - acc: 0.6929
Epoch 4/20
2400/2400 [==============================] - 2s 789us/step - loss: 0.5609 - acc: 0.7037
Epoch 5/20
2400/2400 [==============================] - 2s 785us/step - loss: 0.5453 - acc: 0.7146
Epoch 6/20
2400/2400 [==============================] - 2s 794us/step - loss: 0.5459 - acc: 0.7098
Epoch 7/20
2400/2400 [==============================] - 2s 791us/step - loss: 0.5209 - acc: 0.7288
Epoch 8/20
2400/2400 [==============================] - 2s 784us/step - loss: 0.5286 - acc: 0.7325
Epoch 9/20
2400/2400 [==============================] - 2s 789us/step - loss: 0.5030 - acc: 0.7392
Epoch 10/20
2400/2400 [==============================] - 2s 782us/step - loss: 0.4899 - acc: 0.7510
Epoch 11/20

2400/2400 [==============================] - 3s 1ms/step - loss: 0.6754 - acc: 0.6327
Epoch 2/20
2400/2400 [==============================] - 2s 802us/step - loss: 0.5923 - acc: 0.6808
Epoch 3/20
2400/2400 [==============================] - 2s 806us/step - loss: 0.5653 - acc: 0.7115
Epoch 4/20
2400/2400 [==============================] - 2s 797us/step - loss: 0.5367 - acc: 0.7225
Epoch 5/20
2400/2400 [==============================] - 2s 791us/step - loss: 0.5295 - acc: 0.7350
Epoch 6/20
2400/2400 [==============================] - 2s 799us/step - loss: 0.5329 - acc: 0.7296
Epoch 7/20
2400/2400 [==============================] - 2s 792us/step - loss: 0.4943 - acc: 0.7615
Epoch 8/20
2400/2400 [==============================] - 2s 798us/step - loss: 0.4934 - acc: 0.7565
Epoch 9/20
2400/2400 [==============================] - 2s 795us/step - loss: 0.4729 - acc: 0.7742
Epoch 10/20
2400/2400 [==============================] - 2s 802us/step - loss: 0.4506 - acc: 0.7910
Epoch 11/20
2400/2400 

3000/3000 [==============================] - 2s 824us/step - loss: 0.4715 - acc: 0.7625
Epoch 11/20
3000/3000 [==============================] - 2s 826us/step - loss: 0.4441 - acc: 0.7825
Epoch 12/20
3000/3000 [==============================] - 2s 831us/step - loss: 0.4250 - acc: 0.7892
Epoch 13/20
3000/3000 [==============================] - 2s 832us/step - loss: 0.3970 - acc: 0.8120
Epoch 14/20
3000/3000 [==============================] - 2s 828us/step - loss: 0.3696 - acc: 0.8353
Epoch 15/20
3000/3000 [==============================] - 2s 831us/step - loss: 0.3614 - acc: 0.8338
Epoch 16/20
3000/3000 [==============================] - 2s 831us/step - loss: 0.3313 - acc: 0.8583
Epoch 17/20
3000/3000 [==============================] - 2s 826us/step - loss: 0.3283 - acc: 0.8602
Epoch 18/20
3000/3000 [==============================] - 2s 814us/step - loss: 0.3013 - acc: 0.8753
Epoch 19/20
3000/3000 [==============================] - 3s 835us/step - loss: 0.2693 - acc: 0.8860
Epoch 20/20


In [28]:
bayes_result.cv_results_

defaultdict(list,
            {'split0_test_score': [0.6916666666666667, 0.6608333333333334],
             'split1_test_score': [0.7016666666666667, 0.7033333333333334],
             'split2_test_score': [0.6983333333333334, 0.6983333333333334],
             'split3_test_score': [0.6858333333333333, 0.7416666666666667],
             'split4_test_score': [0.69, 0.6616666666666666],
             'mean_test_score': [0.6935, 0.6931666666666667],
             'std_test_score': [0.005734883511361772, 0.030066592756745825],
             'rank_test_score': [1, 1],
             'mean_fit_time': [40.432296943664554, 22.13529682159424],
             'std_fit_time': [0.4201621995601064, 0.425421316343275],
             'mean_score_time': [0.382991886138916, 0.6271965026855468],
             'std_score_time': [0.060336104509796616, 0.07606775679887931],
             'param_batch_size': [16, 16],
             'param_epochs': [20, 10],
             'param_lr': [0.001, 0.1],
             'params': [{'

In [29]:
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))
means = random_result.cv_results_['mean_test_score']
stds = random_result.cv_results_['std_test_score']
params = random_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.693500 using {'batch_size': 16, 'epochs': 20, 'lr': 0.001}
0.693500 (0.005735) with: {'batch_size': 16, 'epochs': 20, 'lr': 0.001}
0.693167 (0.030067) with: {'batch_size': 16, 'epochs': 10, 'lr': 0.1}
